In [216]:
import re
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from keras.models import Sequential
import numpy as np

In [217]:
with open('classic_poems.json', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [218]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [219]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('oet', 2496), ('id', 2496), ('ushkin', 479), ('title', 2496), ('к', 1064), ('наталье', 1), ('content', 2497), ('ourquoi', 3), ('cr', 5), ('indr', 1)]


In [220]:
data = tokenizer.texts_to_sequences([texts])


In [221]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(171675, 1000)


In [222]:
inp_words = 3
n = res.shape[0]-inp_words

In [223]:
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )
print( res.shape )
 
inp_words = 3
n = res.shape[0]-inp_words
 
X = np.array([res[i:i+inp_words] for i in range(n)])
Y = keras.utils.to_categorical(res[inp_words:], num_classes=maxWordsCount)


(171675,)


In [224]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(512, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 3, 256)            256000    
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 512)               393728    
                                                                 
 dense_13 (Dense)            (None, 1000)              513000    
                                                                 
Total params: 1,162,728
Trainable params: 1,162,728
Non-trainable params: 0
_________________________________________________________________


In [225]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
5365/5365 [==============================] - 63s 11ms/step - loss: 5.3761 - accuracy: 0.1137
Epoch 2/50
5365/5365 [==============================] - 62s 12ms/step - loss: 5.1474 - accuracy: 0.1254
Epoch 3/50
5365/5365 [==============================] - 62s 12ms/step - loss: 5.0238 - accuracy: 0.1285
Epoch 4/50
5365/5365 [==============================] - 61s 11ms/step - loss: 4.9169 - accuracy: 0.1315
Epoch 5/50
5365/5365 [==============================] - 61s 11ms/step - loss: 4.8244 - accuracy: 0.1330
Epoch 6/50
5365/5365 [==============================] - 61s 11ms/step - loss: 4.7518 - accuracy: 0.1342
Epoch 7/50
5365/5365 [==============================] - 61s 11ms/step - loss: 4.6833 - accuracy: 0.1353
Epoch 8/50
5365/5365 [==============================] - 62s 11ms/step - loss: 4.6317 - accuracy: 0.1349
Epoch 9/50
5365/5365 [==============================] - 62s 11ms/step - loss: 4.5796 - accuracy: 0.1369
Epoch 10/50
5365/5365 [==============================] - 61s 11m

In [226]:
def buildPhrase(texts, str_len = 20):
  res = texts
  tokenizer.fit_on_texts([texts])
  data = tokenizer.texts_to_sequences([texts])[0]
  for i in range(str_len):
    x = data[i: i+inp_words]
    inp = np.expand_dims(x, axis=0)
 
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
 
    res += " " + tokenizer.index_word[indx] # дописываем строку
 
  return res

In [227]:
res = buildPhrase("позитив добавляет годы")
print(res)

1/1 [==============================] - 0s 19ms/step
позитив добавляет годы небесный роковой стою мою легкий образ вечно юный тоской туда туда туда туда туда туда туда туда туда туда туда


In [228]:
from random import choice
 
 
def mygen(s: str, n: int):
    for _ in range(n):
        yield choice(s.split())



m=""
for i in range(6):
    
    res = buildPhrase(next(mygen(texts, 1))+" " +next(mygen(texts, 1))+" " +next(mygen(texts, 1)))
    m+=" "+res+" \n "
print(m)

1/1 [==============================] - 0s 21ms/step
 каплями проток звуки.\nОн nвесь нами грусть nтак думал nсердце птица не nчерез его nон славой глядит nс неба светит небу луна nему светит 
  глуби\nНад мой звуки б я с вами я рад n    не ты поэт oet id esenin title content в и ночью content не рука 
  Ни монастырь.\nТам и света и поэта nкак ты nчерез моря nсквозь роз nсквозь ночь сквозь в ночь nя навстречу nи думы nи со 
  и расплескала утешает темно под тенью душу светом nему знать чему свободы вкруг него молча ждет nсквозь неба nдовольно n         в чего нибудь nбудет 
  закрутили,\nНа томиться твоей, nскоро nтвоя nскоро скоро nстоит среди nпод лазури едва весны nи до ночи край луч ль над роковой nи наших 
  зима "Там, обымет\nИ ваши стало кровь равно силой бога nно и ты три время пока след нам nесть право nесли долго ты в 
 
